# Threading

### Esecuzione di una funzione in un thread

In [1]:
import threading


def mia_funzione(arg1, arg2):
    #
    pass

In [2]:
t1 = threading.Thread(target=mia_funzione, args=(1,), kwargs={'arg2': 2})

In [3]:
t1

<Thread(Thread-5, initial)>

In [4]:
t1.name = 'thread 1'  # modifica il nome al thread (default: Thread-n). In alternativa si può anche usare t1.setName(str).

In [5]:
t1.daemon = True  # se True, il thread termina quando termina il main thread (default: False). In alternativa si può anche usare t1.setDaemon(bool).

In [6]:
t1.start()  # avvia il thread

In [7]:
t1

<Thread(thread 1, stopped daemon 6235795456)>

In [8]:
t1.ident  # identificatore del thread in Python

6235795456

In [9]:
t1.native_id  # identificatore del thread nel sistema operativo

4132521

In [10]:
t1.isDaemon()  # True se il thread è un daemon

True

In [11]:
t1.is_alive()  # True se il thread è in esecuzione

False

In [12]:
t1.join()  # aspetta che il thread termini

### Strumenti per la gestione dei thread

In [13]:
def funzione_infinita():
    while True:
        pass


for i in range(3):  # genero tre thread (totale di 4 con il MainThread)
    threading.Thread(target=funzione_infinita).start()

In [14]:
threading.active_count()  # ritorna il numero di thread attivi

11

In [15]:
threading.enumerate()  # ritorna una lista di tutti i thread attivi

[<_MainThread(MainThread, started 8522999040)>,
 <Thread(IOPub, started daemon 6116290560)>,
 <Heartbeat(Heartbeat, started daemon 6133116928)>,
 <Thread(Thread-3, started daemon 6151090176)>,
 <Thread(Thread-4, started daemon 6167916544)>,
 <ControlThread(Control, started daemon 6184742912)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 6201569280)>,
 <ParentPollerUnix(Thread-2, started daemon 6218969088)>,
 <Thread(Thread-6, started 6235795456)>,
 <Thread(Thread-7, started 6252621824)>,
 <Thread(Thread-8, started 6269448192)>]

In [16]:
threading.current_thread()  # ritorna l’oggetto del thread corrente

<_MainThread(MainThread, started 8522999040)>

In [17]:
threading.get_native_id()  # ritorna l’id del thread corrente

4132435

In [18]:
threading.get_ident()  # ritorna l’id del thread corrente

8522999040

Exception Hook

In [19]:
def genera_errore():  # funzione che genera un errore
    raise Exception("Eccezione generata")


def hook_di_eccezione(args):  # funzione che gestisce l’eccezione (callback)
    print(f'Thread Terminato: {args.exc_value}')


threading.excepthook = hook_di_eccezione
threading.Thread(target=genera_errore).start()

Thread Terminato: Eccezione generata


### Funzioni bloccanti

In [20]:
import time

time.sleep(0.1)  # funzione che sospende l’esecuzione del thread per 0.1 secondi

## Primitive di sincronizzazione

## Lock

In [21]:
import threading

lock = threading.Lock()  # creo un oggetto di tipo Lock
lock.acquire()  # acquisisco il lock
# ... eseguo il codice che richiede la mutua esclusione
lock.release()  # rilascio il lock

Alternative di uso di acquire

In [22]:
if 1 != 1:  # non eseguo il codice per evitare di bloccare il MainThread senza rilasciarlo
    lock.acquire(blocking=False)  # se il lock è già acquisito, non aspetta e ritorna False
    # oppure
    lock.acquire(timeout=10)  # se il lock è già acquisito, aspetta al massimo 10 secondi e ritorna False
    # oppure
    with lock:  # acquisisce il lock e lo rilascia automaticamente alla fine del blocco with
        # ... eseguo il codice che richiede la mutua esclusione
        pass

    lock.release()  # rilascio

Esercizio - mutex lock

In [23]:
lock = threading.Lock()


def task():
    with lock:  # acquisisce il lock e lo rilascia automaticamente alla fine del blocco with
        time.sleep(1)  # funzione che sospende l’esecuzione del thread per 1 secondo
        print(f'Il {threading.current_thread().name} ha acquisito la lock. Attendo 1 secondo...')

In [24]:
def main():
    threads = []
    for i in range(10):  # creo 10 thread che eseguono la funzione task
        threads.append(threading.Thread(target=task))

    for t in threads:
        t.start()  # avvio i thread
    for t in threads:
        t.join()  # aspetto che i thread terminino

In [25]:
def task():
    time.sleep(1)  # funzione che sospende l’esecuzione del thread per 1 secondo
    print(f'Il {threading.current_thread().name} ha acquisito la lock. Attendo 1 secondo...')

## RLock

Esercizio - mutex  RLock

In [26]:
def task2(lock):
    with lock:
        print(f'{threading.current_thread().name} ha acquisito la seconda lock.')


def task(lock):
    with lock:  # acquisisce il lock e lo rilascia automaticamente alla fine del blocco with
        time.sleep(1)  # funzione che sospende l’esecuzione del thread per 1 secondo
        print(f'Il {threading.current_thread().name} ha acquisito la lock. Attendo 1 secondo...')
        task2(lock)


lock = threading.RLock()  # creo un oggetto di tipo RLock
threads = []
for i in range(10):  # creo 10 thread che eseguono la funzione task
    threads.append(threading.Thread(target=task, args=(lock,)))

for t in threads:
    t.start()  # avvio i thread
for t in threads:
    t.join()  # aspetto che i thread terminino


Il Thread-10 ha acquisito la lock. Attendo 1 secondo...
Thread-10 ha acquisito la seconda lock.
Il Thread-11 ha acquisito la lock. Attendo 1 secondo...
Thread-11 ha acquisito la seconda lock.
Il Thread-12 ha acquisito la lock. Attendo 1 secondo...
Thread-12 ha acquisito la seconda lock.
Il Thread-13 ha acquisito la lock. Attendo 1 secondo...
Thread-13 ha acquisito la seconda lock.
Il Thread-14 ha acquisito la lock. Attendo 1 secondo...
Thread-14 ha acquisito la seconda lock.
Il Thread-15 ha acquisito la lock. Attendo 1 secondo...
Thread-15 ha acquisito la seconda lock.
Il Thread-16 ha acquisito la lock. Attendo 1 secondo...
Thread-16 ha acquisito la seconda lock.
Il Thread-17 ha acquisito la lock. Attendo 1 secondo...
Thread-17 ha acquisito la seconda lock.
Il Thread-18 ha acquisito la lock. Attendo 1 secondo...
Thread-18 ha acquisito la seconda lock.
Il Thread-19 ha acquisito la lock. Attendo 1 secondo...
Thread-19 ha acquisito la seconda lock.


## Condizione

Esercizio - mutex condition

In [27]:
if 1 != 1:
    condition = threading.Condition()  # creo un oggetto Condition
    condition = threading.Condition(
        lock=threading.Lock())  # creo un oggetto Condition con un oggetto Lock come parametro

    condition.acquire()  # acquisisco il lock
    condition.wait()  # sospende l’esecuzione del thread fino a quando non viene invocato il metodo notify() o notify_all()
    condition.release()  # rilascio il lock
    # oppure
    with condition:  # acquisisce il lock e lo rilascia automaticamente alla fine del blocco with
        condition.wait()  # sospende l’esecuzione del thread fino a quando non viene invocato il metodo notify() o notify_all()
        # ... eseguo il codice che richiede la mutua esclusione
        pass

Esercizio - mutex condition

In [28]:
def task(condizione, risultato):
    time.sleep(1)  # sospendo l’esecuzione del thread per 1 secondo
    risultato.append(1)  # assegno il valore 1 alla variabile risultato
    print('Invio la notifica dal thread...')
    with condizione:
        condizione.notify()


condizione = threading.Condition()  # creo un oggetto Condition
risultato = []

print('MainThread in attesa di notifica...')
with condizione:  # acquisisce il lock e lo rilascia automaticamente alla fine del blocco
    threading.Thread(target=task, args=(condizione, risultato)).start()
    condizione.wait()  # sospende l’esecuzione del thread fino a quando non viene invocato il metodo notify() o notify_all()

print(f'Risultato ottenuto: {risultato}')  # stampo il risultato ottenuto

MainThread in attesa di notifica...
Invio la notifica dal thread...
Risultato ottenuto: [1]


## Semaforo

In [29]:
semaphore = threading.Semaphore(10)  # 10 threads possono accedere alla sezione critica
semaphore.acquire()  # decrementa il contatore
# ... eseguo il codice che richiede la mutua esclusione
semaphore.release()  # incrementa il contatore

In [30]:
if 1 != 1:
    semaphore.acquire(blocking=False)  # decrementa il contatore se il contatore è > 0, altrimenti ritorna False
    # oppure
    semaphore.acquire(
        timeout=10)  # decrementa il contatore se il contatore è > 0, altrimenti ritorna False dopo 10 secondi
    # oppure
    with semaphore:  # decrementa il contatore e lo incrementa alla fine del blocco with
        # ... eseguo il codice che richiede la mutua esclusione
        pass

Esercizio - mutex semaphore

In [31]:
semaforo = threading.Semaphore(2)


def task():
    with semaforo:  # decrementa il contatore e lo incrementa alla fine del blocco with
        time.sleep(1)  # funzione che sospende l’esecuzione del thread per 1 secondo
        print(f'Il {threading.current_thread().name} ha acquisito il semaforo. Attendo 1 secondo...')


threads = []
for i in range(10):  # creo 10 thread che eseguono la funzione task
    threads.append(threading.Thread(target=task))

for t in threads:
    t.start()  # avvio i thread
for t in threads:
    t.join()  # aspetto che i thread terminino

Il Thread-22 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-21 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-24 ha acquisito il semaforo. Attendo 1 secondo...Il Thread-23 ha acquisito il semaforo. Attendo 1 secondo...

Il Thread-26 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-25 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-27 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-28 ha acquisito il semaforo. Attendo 1 secondo...
Il Thread-30 ha acquisito il semaforo. Attendo 1 secondo...Il Thread-29 ha acquisito il semaforo. Attendo 1 secondo...



## Evento

In [32]:
evento = threading.Event()  # creo un oggetto di tipo Event
evento.isSet()  # ritorna True se l’evento è impostato, False altrimenti
evento.set()  # imposto l’evento
evento.clear()  # resetto l’evento
# evento.wait()  # sospende l’esecuzione del thread fino a quando l’evento non viene impostato
# evento.wait(timeout=10)  # sospende l’esecuzione del thread fino a quando l’evento non viene impostato o scade il timeout

Esercizio - mutex event

In [33]:
evento = threading.Event()  # creo un oggetto di tipo Event


def task():
    print(f'Il {threading.current_thread().name} attende l\’evento...')
    evento.wait()  # sospende l’esecuzione del thread fino a quando l’evento non viene impostato
    print(f"L’evento è stato impostato. Il {threading.current_thread().name} continua l’esecuzione...")


threads = []
for i in range(10):  # creo 10 thread che eseguono la funzione task
    threads.append(threading.Thread(target=task))

for t in threads:
    t.start()  # avvio i thread

time.sleep(2)  # sospendo l’esecuzione del thread principale per 2 secondi
evento.set()  # imposto l’evento

for t in threads:
    t.join()  # aspetto che i thread terminino

Il Thread-31 attende l\’evento...Il Thread-32 attende l\’evento...
Il Thread-33 attende l\’evento...
Il Thread-34 attende l\’evento...

Il Thread-35 attende l\’evento...
Il Thread-36 attende l\’evento...
Il Thread-37 attende l\’evento...
Il Thread-38 attende l\’evento...
Il Thread-39 attende l\’evento...
Il Thread-40 attende l\’evento...
L’evento è stato impostato. Il Thread-32 continua l’esecuzione...L’evento è stato impostato. Il Thread-34 continua l’esecuzione...

L’evento è stato impostato. Il Thread-39 continua l’esecuzione...
L’evento è stato impostato. Il Thread-37 continua l’esecuzione...
L’evento è stato impostato. Il Thread-31 continua l’esecuzione...
L’evento è stato impostato. Il Thread-36 continua l’esecuzione...
L’evento è stato impostato. Il Thread-33 continua l’esecuzione...
L’evento è stato impostato. Il Thread-35 continua l’esecuzione...
L’evento è stato impostato. Il Thread-38 continua l’esecuzione...
L’evento è stato impostato. Il Thread-40 continua l’esecuzione...


## Timer

In [ ]:
# creo un oggetto di tipo Timer che esegue la funzione print dopo 2 secondi
timer = threading.Timer(interval=2, function=print, args=('Ciao Mondo!',))
timer.start()  # avvio il timer
timer.cancel()  # annulla il timer

Esercizio - timer

In [34]:
def task(messaggio):
    print(messaggio)


# creo un oggetto di tipo Timer che esegue la funzione task dopo 3 secondi
timer = threading.Timer(0.3, task, args=('Ciao Mondo!',))

timer.start()  # avvio il timer
print('Il timer è stato avviato. Attendo 0.3 secondi prima della sua esecuzione...')
timer.join()

Il timer è stato avviato. Attendo 3 secondi prima della sua esecuzione...
Ciao Mondo!


## Barriera

In [35]:
if 1 != 1:
    barriera = threading.Barrier(10)  # creo un oggetto di tipo Barrier che attende massimo 10 thread
    # oppure
    barriera = threading.Barrier(10, action=funzione_di_callback)  # creo un oggetto di tipo Barrier che attende massimo 10 thread ed esegue la funzione funzione_di_callback quando tutti i thread hanno raggiunto la barriera
    # oppure
    barriera = threading.Barrier(10, timeout=5)  # creo un oggetto di tipo Barrier che attende massimo 10 thread e scade dopo 5 secondi

    barriera.wait()  # sospende l’esecuzione del thread fino a quando tutti i thread non hanno raggiunto la barriera

Esercizio - mutex barrier

In [36]:
import random


def task(barriera):
    time.sleep(random.randint(1, 3)/10)  # sospendo l’esecuzione del thread per un tempo casuale tra 0.1 e 0.3 secondi
    print(f'Il {threading.current_thread().name} ha raggiunto la barriera.')
    barriera.wait()  # sospende l’esecuzione del thread fino a quando tutti i thread non hanno raggiunto la barriera
    print(f'Il {threading.current_thread().name} ha superato la barriera.')


barriera = threading.Barrier(6)  # creo un oggetto di tipo Barrier che attende massimo 6 thread
threads = []
for i in range(6):  # creo 6 thread che eseguono la funzione task
    threads.append(threading.Thread(target=task, daemon=True, args=(barriera,)))

for t in threads:
    t.start()  # avvio i thread

for t in threads:
    t.join()  # aspetto che i thread terminino

Il Thread-42 ha raggiunto la barriera.
Il Thread-45 ha raggiunto la barriera.
Il Thread-44 ha raggiunto la barriera.
Il Thread-43 ha raggiunto la barriera.
Il Thread-47 ha raggiunto la barriera.
Il Thread-46 ha raggiunto la barriera.
Il Thread-46 ha superato la barriera.
Il Thread-44 ha superato la barriera.
Il Thread-43 ha superato la barriera.
Il Thread-45 ha superato la barriera.
Il Thread-47 ha superato la barriera.
Il Thread-42 ha superato la barriera.
